In [1]:
import sys
sys.path.append('../src')

import pandas as pd
import mention_extraction.extract_mentions as eim
import mention_extraction.drug_ici_config
import json

JSON file created successfully.


In [2]:
config_file_json = "drug_symptom_dicts.json"
def load_config(config_file_json):
    with open(config_file_json, 'r') as f:
        config = json.load(f)
    return config
config = load_config(config_file_json)

In [3]:
notes = pd.read_csv('../data/mock_ehr.csv')

In [4]:
notes

,patient_index,report_id,notes
0,P001,R001,Patient with metastatic melanoma currently rec...
1,P002,R002,The patient has stage IV non-small cell lung c...
2,P003,R003,History of renal cell carcinoma on combination...
3,P004,R004,Metastatic urothelial carcinoma treated with a...
4,P005,R005,Patient with head and neck squamous cell carci...
5,P006,R006,Melanoma patient on pembrolizumab reports no n...
6,P007,R007,Stage IV small cell lung cancer patient receiv...
7,P008,R008,Renal cell carcinoma on ipilimumab monotherapy...
8,P009,R009,Metastatic melanoma on atezolizumab. Mild eryt...
9,P010,R010,NSCLC patient treated with pembrolizumab. Repo...


In [5]:
drug_mentions = eim.extract_ici_mentions(notes, config['drug_dict'])
symptom_mentions = eim.extract_symptom_mentions(notes, config['symptom_dict'])

100%|██████████| 10/10 [00:00<00:00, 1370.87it/s]


In [6]:
symptom_mentions[['report_id', 'standard_symptom', 'matched_variant',
       'start_index', 'variant_length', 'theword',
       'symptom_adjacent']].to_csv('../data/symptom_mentions.csv', index=False)
drug_mentions[['report_id', 'standard_drug', 'matched_variant', 'start_index',
       'variant_length', 'theword', 'drug_adjacent']].to_csv('../data/drug_mentions.csv', index=False)

In [7]:
drug_part = drug_mentions.groupby(['report_id', 'notes', 'standard_drug'], as_index=False).agg({'drug_adjacent':list})
symptom_part = symptom_mentions.groupby(['report_id', 'notes', 'standard_symptom'], as_index=False).agg({'symptom_adjacent':list})

In [9]:
final_data = drug_part.merge(symptom_part, on=['report_id', 'notes'], how='inner')
final_data = final_data.merge(notes[['report_id','patient_index']], on = 'report_id', how='inner')

In [10]:
final_data

,report_id,notes,standard_drug,drug_adjacent,standard_symptom,symptom_adjacent,patient_index
0,R001,Patient with metastatic melanoma currently rec...,nivolumab,[Patient with metastatic melanoma currently re...,Diarrhea,[Patient with metastatic melanoma currently re...,P001
1,R001,Patient with metastatic melanoma currently rec...,nivolumab,[Patient with metastatic melanoma currently re...,Fatigue,[Patient with metastatic melanoma currently re...,P001
2,R001,Patient with metastatic melanoma currently rec...,nivolumab,[Patient with metastatic melanoma currently re...,Rash,[Patient with metastatic melanoma currently re...,P001
3,R002,The patient has stage IV non-small cell lung c...,pembrolizumab,[The patient has stage IV non-small cell lung ...,Colitis,[The patient has stage IV non-small cell lung ...,P002
4,R002,The patient has stage IV non-small cell lung c...,pembrolizumab,[The patient has stage IV non-small cell lung ...,Diarrhea,[The patient has stage IV non-small cell lung ...,P002
5,R002,The patient has stage IV non-small cell lung c...,pembrolizumab,[The patient has stage IV non-small cell lung ...,Fatigue,[The patient has stage IV non-small cell lung ...,P002
6,R003,History of renal cell carcinoma on combination...,ipilimumab,[History of renal cell carcinoma on combinatio...,Pruritus,[History of renal cell carcinoma on combinatio...,P003
7,R003,History of renal cell carcinoma on combination...,ipilimumab,[History of renal cell carcinoma on combinatio...,Rash,[History of renal cell carcinoma on combinatio...,P003
8,R003,History of renal cell carcinoma on combination...,ipilimumab,[History of renal cell carcinoma on combinatio...,Shortness of Breath,[History of renal cell carcinoma on combinatio...,P003
9,R003,History of renal cell carcinoma on combination...,nivolumab,[History of renal cell carcinoma on combinatio...,Pruritus,[History of renal cell carcinoma on combinatio...,P003


In [11]:
final_data.to_csv('../data/final_data.csv', index=False)


# Appendix